In [1]:
import numpy as np
# np.seterr(all='raise')
np.seterr(under='ignore', invalid='raise', divide='raise', over='raise')
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.special import kv  # K_v BESSEL FUNCTIONS
from scipy.special import zeta, polygamma, factorial
import matplotlib as mpl
from mpmath import zeta
from itertools import cycle
from functions import m_sv_interp, gstar_interp
from scipy.special import kve

mpl.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "font.size": 12,
})


In [ ]:
Mpl = 1.2209e19
GSTAR = 90.0
GSTAR_S = 90.0

path = "/Users/charlottemyers/projects/ctp/heffBW.dat"
data = np.loadtxt(path)
T_data = data[:,0]
g_eff = data[:,1]
def gstar_interp(T):
    return np.interp(T, T_data, g_eff)

In [ ]:
#####
# CONSTANTS
#######
# ----------------------------
# Constant values
# ----------------------------
GSTAR = 90.0
GSTAR_S = 90.0
ME = 0.000511e-3  # GeV
ALPHA_EM = 1.0/137.0
MPL   = 1.2209e19      # GeV
INCLUDE_HS_IN_H = True

# --- unit conversions: cm^3/s -> GeV^-2 ---
cm_to_GeV_inv = 5.0677307e13
s_to_GeV_inv  = 1.5192674e24
cm3s_to_GeV2  = (cm_to_GeV_inv**3) / s_to_GeV_inv
Gev2_to_cm3s = 1.0 / cm3s_to_GeV2

In [4]:
#####
# Model parameters and functions
#######

# ----------------------------
# Functions for cross sections
# ----------------------------
def gD_of_alpha(alphaD):           # g_D from alpha_D
    return np.sqrt(4.0*np.pi*alphaD)

def sigma_xx_to_AA(alphaD, mchi, mA):
    gD = gD_of_alpha(alphaD)
    r = mA / mchi
    amplitude = 16*gD**4 * (1 - r**2)/(9 * (r**2 - 2)**2)
    phase_space = 9 * np.sqrt(1 - r**2) / (64* np.pi * mchi**2)
    return amplitude * phase_space

def sigmav_chichi_ee(epsilon, alphaD, mchi, mA):
    # source:1702.07716 (app)
    if mchi <= ME:
        return 0.0 # kinematically forbidden
    r   = mA / mchi
    e2  = 4.0*np.pi*ALPHA_EM
    gD  = gD_of_alpha(alphaD)
    pref = 4.0 * e2 * (epsilon**2) * (gD**2)
    num  = (2.0 + (ME**2)/(mchi**2))
    den  = (r**2 - 4.0)**2

    phase_space  = np.sqrt(1.0 - (ME**2)/(mchi**2)) / (8.0*np.pi*mchi**2)
    return pref * (num/den) * phase_space

def gammaA_ee(epsilon, mA):
    pref = epsilon**2 * ALPHA_EM / 3.0
    return pref * mA * (1 + 2*(ME**2)/(mA**2)) * np.sqrt(1 - 4*(ME**2)/(mA**2))

# ----------------------------
# ----------------------------
# Model parameters
# ----------------------------
